In [2054]:
#import libraries
import nltk
import random
from nltk.chat.util import Chat, reflections
import math
import numpy as np
import pandas as pd
import json
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.tree import DecisionTreeClassifier
from nltk import WordNetLemmatizer
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk import FreqDist
from chatbot import Chatbot

pd.set_option("display.max_rows", 200)
pd.set_option("display.max_columns", None)

random.seed(7) #set the seed

In [2055]:
def import_data(): #imports dataset as ["category", "user query", "chatbot response"]
    with open("data.json") as intent: #load jsons
        data = json.load(intent)
    df = pd.DataFrame(data['intents'])
    df2 = df[['intent', "text", "responses"]] #choose specific columns
    df2 = df2.rename(columns={"text":"in", "responses":"out"}) #making it more readable

    return df2

In [2056]:
def get_tree():
    t = DecisionTreeClassifier(
        criterion="gini", 
        splitter="best",
        max_depth = 4,
        min_samples_leaf = 100,
        min_samples_split = 1000,
        random_state=1)
    
    df = import_data()
    return df

In [2057]:
df = import_data()
df["intent"] = df["intent"].apply(lambda x: str.lower(x))
df["in"] = df["in"].apply(lambda x: [str.lower(n) for n in x])
# df["out"] = df["out"].apply(lambda x: [str.lower(n) for n in x])
df.head(5)

,intent,in,out
0,greeting,"[hi, hi there, hola, hello, hello there, hya, ...","[Hi human, , Hello human, , Hola human, ]"
1,greetingresponse,"[my name is chatbot, this is chatbot, i am cha...","[Great! Hi! How can I help?, Good! Hi, how can..."
2,courtesygreeting,"[how are you?, hi how are you?, hello how are ...","[Hello, I am great, how are you? , Hello, how ..."
3,courtesygreetingresponse,"[good thanks! my name is chatbot, good thanks!...","[Great! Hi! How can I help?, Good! Hi, how can..."
4,currenthumanquery,"[what is my name?, what do you call me?, who d...","[You are! How can I help?, Your name is , how ..."


In [2058]:
temp = pd.DataFrame(columns=["intent", "in", "out"])
for index, row in df.iterrows():
    for i in row["in"]:
        temp.loc[len(temp.index)] = [row["intent"], i, row["out"]]
df = temp.sort_values("intent")
df = df.reset_index()
df.drop("index", axis=1, inplace=True)
df.head(5)

,intent,in,out
0,clever,you are a very clever girl,"[Thank you, I was trained that way, I was trai..."
1,clever,genious,"[Thank you, I was trained that way, I was trai..."
2,clever,clever girl,"[Thank you, I was trained that way, I was trai..."
3,clever,you are very clever,"[Thank you, I was trained that way, I was trai..."
4,clever,you are a very intelligent girl,"[Thank you, I was trained that way, I was trai..."


In [2059]:
cvin = []
for i in df["in"]:
    cvin.append(i)

cv = CountVectorizer()
cvout = cv.fit_transform(cvin).toarray()
cvwords = cv.get_feature_names_out()

cvdf = pd.DataFrame(columns=cvwords, data=cvout)

df2 = pd.concat([df["intent"], cvdf], axis=1)
df2.head(5)

,intent,about,adios,am,any,anyone,are,aware,bay,be,bored,by,bye,call,camera,can,chatbot,chatbotous,cheering,clever,communicating,comprendo,conscious,could,do,doing,door,enough,for,friends,fuck,genious,get,girl,give,good,goodbye,gossip,got,great,have,hear,hello,help,helpful,hi,hola,hope,how,hya,identify,in,intelligent,is,it,joke,jokes,know,later,laugh,make,me,mean,meant,more,my,name,need,not,off,ok,open,please,pod,prove,quiet,real,saying,see,self,shhh,shit,shut,some,speaking,stop,surely,talking,tell,thank,thanks,that,the,there,think,this,time,to,twat,understand,up,very,want,was,wasn,well,what,who,why,will,with,you,your
0,clever,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0
1,clever,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,clever,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,clever,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0
4,clever,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0


In [2060]:
cb = Chatbot()

def get_intent(string): #string = user query
    arr = cb.lemmatize(cb.tokenize(string))
    highscore = 0
    intent = ""
    for word in arr:
        for i, row in df.iterrows():
            score = 0
            try:
                if df.loc[i, word] == 1:
                    score += 1
            except:
                pass
            if score > highscore:
                intent = row["intent"]
    print(intent)
    print(df.iloc[df["intent"] == intent, 2])

In [2061]:
x = df2.drop("intent", axis=1)
y = df2["intent"]

In [2062]:
xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size=0.3, random_state=1)

In [2063]:
#make a model and fit to data
model = DecisionTreeClassifier()
model.fit(xtrain, ytrain)

DecisionTreeClassifier()

In [2064]:
model.score(xtest, ytest)

0.5581395348837209

In [2065]:
def predict(string): #string = "user input"
    #output: array for decision tree
    arr = string.split(" ")

    temp = pd.DataFrame(columns=cvwords, data=[[0 for x in cvwords]])
    i = 0
    for col in temp.columns:
        for i in arr:
            if col == i:
                temp[col] = 1
    return model.predict(temp)[0]

def respond(string): #string = "user input"
    intent = predict(string)
    try:
        responses = df.loc[df["intent"] == intent, "out"]
        r = random.randint(0, responses.shape[0] - 1)
        choice = responses.iloc[r][0]
        return choice
    except:
        return "Sorry, I don't understand."

In [2066]:
respond("do you harbor hatred towards minorities?")

"I met a Dutch girl with inflatable shoes last week, phoned her up to arrange a date but unfortunately she'd popped her clogs.  "

In [2067]:
'''
TODO:
- turn all columns from count vectorizer into lemmas, do the same for predict()
- add "other" intent as default (if the GINI on the prediction is above 0.4 or something)

- WHEN ALL OF THAT IS DONE, put all of this sequentially into test.py and import the respond() function into chatbot.py
'''

'\nTODO:\n- turn all columns from count vectorizer into lemmas, do the same for predict()\n- add "other" intent as default (if the GINI on the prediction is above 0.4 or something)\n'